In [15]:
import numpy as np
import pandas as pd
from list1.algorithms.pbil import pbil
from list1.utils import random_population

def f_one_max(x: np.ndarray):
    return np.sum(x)


def f_deceptive_one_max(x: np.ndarray):
    d = len(x)
    ones_count = np.sum(x)

    if ones_count == 0:
        return d + 1
    return ones_count


def f_k_deceptive_one_max(x: np.ndarray, k):
    d = len(x)
    res = 0
    for i in range(0, d, k):
        r = min(d, i + k)
        total = r - i
        sub_x = x[i:r]

        ones_count = np.sum(sub_x == 1)
        if ones_count == 0:
            res += total + 1
        else:
            res += ones_count

    return res

In [9]:
def run_once(d, pop_size, learn_rate,
             mut_prob, mut_shift, max_iter,
             sample_size=100):
    rng = np.random.default_rng()
    prob_vector = pbil(
        d=d,
        eval_f=f_one_max,
        pop_size=pop_size,
        learn_rate=learn_rate,
        mut_prob=mut_prob,
        mut_shift=mut_shift,
        max_iter=max_iter,
        rng=rng,
    )
    sample = random_population(prob_vector, sample_size, rng)
    scores = np.array([f_one_max(x) for x in sample])
    return int(scores.max()), float(scores.mean())


def onemax(
        ds=(50, 100),
        pop_sizes=(5, 10, 20),
        learn_rates=(0.05, 0.1, 0.2),
        mut_probs=(0.0, 0.01, 0.02),
        mut_shifts=(0.05,),
        max_iter=200,
        repeats=20,
):
    rows = []
    for d in ds:
        opt = d
        for N in pop_sizes:
            for lr in learn_rates:
                for mp in mut_probs:
                    for ms in mut_shifts:
                        successes = 0
                        bests, means = [], []
                        for _ in range(repeats):
                            best, mean = run_once(d, N, lr, mp, ms, max_iter)
                            successes += int(best == opt)
                            bests.append(best)
                            means.append(mean)
                        rows.append({
                            "d": d, "N": N, "learn_rate": lr, "mut_prob": mp, "mut_shift": ms, "iters": max_iter,
                            "success_rate": successes / repeats,
                            "avg_best": float(np.mean(bests)),
                            "avg_mean": float(np.mean(means)),
                        })
    return rows

In [20]:

results = onemax(
    ds=(20, 50, 100),
    pop_sizes=(5, 20, 100),
    learn_rates=(0.05, 0.1, 0.2),
    mut_probs=(0.0, 0.01, 0.02),
    mut_shifts=(0.05,),
    max_iter=2000,
    repeats=5,
)

In [21]:
df = pd.DataFrame(results)
df = df[['d','N','learn_rate','mut_prob','mut_shift','iters','success_rate','avg_best','avg_mean']]
print(df.to_string(index=False))

  d   N  learn_rate  mut_prob  mut_shift  iters  success_rate  avg_best  avg_mean
 20   5        0.05      0.00       0.05   2000           1.0      20.0    20.000
 20   5        0.05      0.01       0.05   2000           1.0      20.0    19.896
 20   5        0.05      0.02       0.05   2000           1.0      20.0    19.837
 20   5        0.10      0.00       0.05   2000           1.0      20.0    20.000
 20   5        0.10      0.01       0.05   2000           1.0      20.0    19.949
 20   5        0.10      0.02       0.05   2000           1.0      20.0    19.903
 20   5        0.20      0.00       0.05   2000           1.0      20.0    20.000
 20   5        0.20      0.01       0.05   2000           1.0      20.0    19.977
 20   5        0.20      0.02       0.05   2000           1.0      20.0    19.964
 20  20        0.05      0.00       0.05   2000           1.0      20.0    20.000
 20  20        0.05      0.01       0.05   2000           1.0      20.0    19.871
 20  20        0